In [1]:
!pip install gurobipy

In [2]:
from gurobipy import *

In [22]:
# nと初期配置を変更しなければいけない
ship_num=5
stack_num=5
height=5
time = 10

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
T=[i+1 for i in range(time)]
n1=[3,2,2,3,3]
# n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O

In [23]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,10,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=20
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

C:\Users\Owner\AppData\Local\Temp\ipykernel_3816\522717571.py:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [24]:
a=[]
d={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  d[tuple(j)]=O.count(j)

d


{(2, 1, 3, 5, 4): 2,
 (2, 1, 3, 4, 5): 4,
 (2, 1, 4, 3, 5): 1,
 (1, 2, 3, 5, 4): 1,
 (3, 1, 4, 2, 5): 1,
 (1, 2, 3, 4, 5): 7,
 (1, 3, 2, 4, 5): 2,
 (1, 2, 4, 3, 5): 1,
 (1, 3, 4, 2, 5): 1}

In [25]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for s in S:
  for h in range(2,len(H)):
    for i in range(len(O)):
      m.addConstr(z[s,h,i]<=z[s,h+1,i])

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

In [26]:
# 初期配置の制約

m.addConstr(x[1,1,O1[4],1]==1)
m.addConstr(x[1,2,O1[2],1]==1)
m.addConstr(x[1,3,O1[0],1]==1)
# m.addConstr(x[1,4,O1[3],1]==1)
m.addConstr(x[2,1,O1[4],1]==1)
m.addConstr(x[2,2,O1[3],1]==1)
# m.addConstr(x[2,3,O1[3],1]==1)
# m.addConstr(x[2,4,O1[1],1]==1)
m.addConstr(x[3,1,O1[3],1]==1)
m.addConstr(x[3,2,O1[3],1]==1)
m.addConstr(x[3,3,O1[0],1]==1)
# m.addConstr(x[3,4,O1[2],1]==1)
# m.addConstr(x[3,5,O1[0],1]==1)
m.addConstr(x[4,1,O1[4],1]==1)
m.addConstr(x[4,2,O1[1],1]==1)
m.addConstr(x[4,3,O1[2],1]==1)
# m.addConstr(x[4,4,O1[2],1]==1)
m.addConstr(x[5,1,O1[1],1]==1)
m.addConstr(x[5,2,O1[0],1]==1)
# m.addConstr(x[6,1,O1[7],1]==1)
# m.addConstr(x[6,2,O1[4],1]==1)
# m.addConstr(x[6,3,O1[6],1]==1)
# m.addConstr(x[6,4,O1[3],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [27]:
beta=0.8
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))

In [28]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 5815 rows, 1851 columns and 38948 nonzeros
Model fingerprint: 0x195a2e75
Variable types: 421 continuous, 1430 integer (1430 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 2157 rows and 517 columns
Presolve time: 0.08s
Presolved: 3658 rows, 1334 columns, 24038 nonzeros
Variable types: 189 continuous, 1145 integer (1145 binary)
Found heuristic solution: objective 6.5000600

Root relaxation: objective 2.117647e-05, 3072 iterations, 0.29 seconds (0.61 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00002    0  253    6.50006    0.00002   100%     -    1s
H    0     0    

In [29]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

x[ 1, 1, 5, 1]=1.0
x[ 1, 2, 3, 1]=1.0
x[ 1, 3, 1, 1]=1.0
x[ 1, 3, 3, 4]=1.0
x[ 2, 1, 5, 1]=1.0
x[ 2, 2, 4, 1]=1.0
x[ 3, 1, 4, 1]=1.0
x[ 3, 2, 4, 1]=1.0
x[ 3, 3, 1, 1]=1.0
x[ 3, 4, 1, 2]=1.0
x[ 3, 5, 1, 3]=1.0
x[ 4, 1, 5, 1]=1.0
x[ 4, 2, 2, 1]=1.0
x[ 4, 3, 2, 5]=1.0
x[ 4, 3, 3, 1]=1.0
x[ 5, 1, 2, 1]=1.0
x[ 5, 1, 4, 6]=1.0
x[ 5, 2, 1, 1]=1.0
y[ 1, 3, 2]=1.0
y[ 2, 5, 5]=1.0
y[ 4, 1, 3]=1.0
y[ 5, 3, 1]=1.0
y[ 5, 4, 4]=1.0
the objective function 5e-05
VaR= 0.0


## 6×6

In [30]:
# nと初期配置を変更しなければいけない
ship_num=8
stack_num=6
height=6
time = 10

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
T=[i+1 for i in range(time)]
n1=[3,3,3,4,3,2,2,3]
# n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O

In [31]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,2*ship_num,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=5
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

C:\Users\Owner\AppData\Local\Temp\ipykernel_3816\1504540335.py:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [32]:
a=[]
d={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  d[tuple(j)]=O.count(j)

d


{(2, 1, 3, 4, 5, 6, 8, 7): 1,
 (1, 2, 3, 5, 4, 6, 8, 7): 1,
 (1, 2, 3, 4, 5, 6, 7, 8): 1,
 (1, 2, 4, 3, 6, 5, 8, 7): 1,
 (1, 2, 3, 4, 6, 5, 7, 8): 1}

In [33]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for s in S:
  for h in range(2,len(H)):
    for i in range(len(O)):
      m.addConstr(z[s,h,i]<=z[s,h+1,i])

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

In [34]:
# 初期配置の制約

m.addConstr(x[1,1,O1[7],1]==1)
m.addConstr(x[1,2,O1[6],1]==1)
m.addConstr(x[1,3,O1[4],1]==1)
m.addConstr(x[1,4,O1[2],1]==1)
m.addConstr(x[1,5,O1[1],1]==1)
m.addConstr(x[2,1,O1[7],1]==1)
m.addConstr(x[2,2,O1[5],1]==1)
m.addConstr(x[2,3,O1[2],1]==1)
m.addConstr(x[2,4,O1[1],1]==1)
m.addConstr(x[3,1,O1[7],1]==1)
m.addConstr(x[3,2,O1[2],1]==1)
m.addConstr(x[3,3,O1[3],1]==1)
m.addConstr(x[3,4,O1[0],1]==1)
m.addConstr(x[4,1,O1[6],1]==1)
m.addConstr(x[4,2,O1[4],1]==1)
m.addConstr(x[4,3,O1[3],1]==1)
m.addConstr(x[4,4,O1[0],1]==1)
m.addConstr(x[5,1,O1[5],1]==1)
m.addConstr(x[5,2,O1[3],1]==1)
m.addConstr(x[6,1,O1[4],1]==1)
m.addConstr(x[6,2,O1[3],1]==1)
m.addConstr(x[6,3,O1[1],1]==1)
m.addConstr(x[6,4,O1[0],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [35]:
beta=0.8
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))

In [36]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 9161 rows, 3306 columns and 79173 nonzeros
Model fingerprint: 0xad7a5c5b
Variable types: 156 continuous, 3150 integer (3150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2220 rows and 796 columns
Presolve time: 0.14s
Presolved: 6941 rows, 2510 columns, 57996 nonzeros
Variable types: 151 continuous, 2359 integer (2359 binary)

Root relaxation: objective 2.489178e-05, 9364 iterations, 1.94 seconds (4.42 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00002    0  667          -    0.00002      -     -    2s
H    0     0                      13.0000700    0.00002   100%

In [39]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  # for (s,h,p,t) in x:
    # if t==1:
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    # if t>=2:
    #   if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

y[ 1, 2, 6]=1.0
y[ 1, 3, 9]=1.0
y[ 1, 5, 7]=1.0
y[ 1, 6, 8]=1.0
y[ 4, 3, 1]=1.0
y[ 4, 5, 2]=1.0
y[ 6, 2, 4]=1.0
y[ 6, 4, 3]=1.0
y[ 6, 5, 5]=1.0
z[ 3, 3, 0]=1.0
z[ 3, 3, 1]=1.0
z[ 3, 3, 2]=1.0
z[ 3, 3, 4]=1.0
z[ 3, 4, 0]=1.0
z[ 3, 4, 1]=1.0
z[ 3, 4, 2]=1.0
z[ 3, 4, 4]=1.0
z[ 3, 5, 0]=1.0
z[ 3, 5, 1]=1.0
z[ 3, 5, 2]=1.0
z[ 3, 5, 4]=1.0
z[ 3, 6, 0]=1.0
z[ 3, 6, 1]=1.0
z[ 3, 6, 2]=1.0
z[ 3, 6, 3]=1.0
z[ 3, 6, 4]=1.0
z[ 5, 5, 3]=1.0
z[ 5, 6, 3]=1.0
the objective function 4.00009
VaR= 4.0


In [43]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    # if t==1:
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t==time:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

#   for (s,k,t) in y:
#     if y[s,k,t].X>EPS:
#       print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

#   for (s,h,i) in z:
#     if z[s,h,i].X>EPS:
#       print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

x[ 1, 1, 8,10]=1.0
x[ 2, 1, 8,10]=1.0
x[ 2, 2, 6,10]=1.0
x[ 2, 3, 3,10]=1.0
x[ 2, 4, 2,10]=1.0
x[ 2, 5, 2,10]=1.0
x[ 2, 6, 2,10]=1.0
x[ 3, 1, 8,10]=1.0
x[ 3, 2, 3,10]=1.0
x[ 3, 3, 4,10]=1.0
x[ 3, 4, 1,10]=1.0
x[ 3, 5, 1,10]=1.0
x[ 3, 6, 7,10]=1.0
x[ 4, 1, 7,10]=1.0
x[ 4, 2, 5,10]=1.0
x[ 4, 3, 1,10]=1.0
x[ 5, 1, 6,10]=1.0
x[ 5, 2, 4,10]=1.0
x[ 5, 3, 4,10]=1.0
x[ 5, 4, 4,10]=1.0
x[ 5, 5, 3,10]=1.0
x[ 6, 1, 5,10]=1.0
x[ 6, 2, 5,10]=1.0
the objective function 4.00009
VaR= 4.0


In [47]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
#   for (s,h,p,t) in x:
    # if t==1:
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    # if t==time:
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

#   for (s,k,t) in y:
#     if y[s,k,t].X>EPS:
#       print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

z[ 3, 3, 0]=1.0
z[ 3, 3, 1]=1.0
z[ 3, 3, 2]=1.0
z[ 3, 3, 4]=1.0
z[ 3, 4, 0]=1.0
z[ 3, 4, 1]=1.0
z[ 3, 4, 2]=1.0
z[ 3, 4, 4]=1.0
z[ 3, 5, 0]=1.0
z[ 3, 5, 1]=1.0
z[ 3, 5, 2]=1.0
z[ 3, 5, 4]=1.0
z[ 3, 6, 0]=1.0
z[ 3, 6, 1]=1.0
z[ 3, 6, 2]=1.0
z[ 3, 6, 3]=1.0
z[ 3, 6, 4]=1.0
z[ 5, 5, 3]=1.0
z[ 5, 6, 3]=1.0
the objective function 4.00009
VaR= 4.0
